<a href="https://colab.research.google.com/github/SrivenkateswaraRao/Research-Project/blob/master/Pest_Classifier_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from os import listdir
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [ ]:
import tensorflow as tf 
import numpy as np


In [ ]:
import tensorflow.keras as ks 

# import the necessary packages


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt

In [ ]:
from google.colab import files
files.upload() 

In [ ]:
import CreatePestImageData as Imagedataset
import SupportingFunctions as support

In [ ]:
support.DownLoadFiles('1aXb25Wp9sG7mL6AQgYY9ElrGcERi95JU',drive)

In [ ]:
support.Unzip('/content/GREENPATROL_DB_V3.0.zip','/content/Tomato/Images')

In [ ]:
support.fileCount('/content/Tomato/Images')

In [ ]:
RemoveFiles('/content/Pest/Labels')

In [ ]:
import CreatePestImageData as Imagedataset

# Data PreProcessing and Transformation 

In [ ]:
class_object = {0 : 'egg_wf' ,
                1 : 'egg_bt', 2 : 'egg_ta', 3 : 'wf', 4 : 'bt', 5 : 'ta', 6 : 'ta_maps', 7 : 'ta_larvae', 8 : 'ta_maps+larvae', 9 : 'tomato'}

In [ ]:
dataSet = Imagedataset.createImageDataset()

In [ ]:
 dataSet.prepare_dataset('/content/Tomato/Labels' ,'/content/Tomato/Images/', class_object,   test_size=8000)

In [ ]:
(trainY , trainX) , (testY , testX ) = (dataSet.trainY , dataSet.trainX ) , (dataSet.testY , dataSet.testX )

In [ ]:
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

In [ ]:
def PrepareAugmentedData(trainX,trainY):
    arr1  = np.fliplr(trainX)
    resultX = np.concatenate((trainX,arr1), axis=0)
    resultY = np.concatenate((trainY,trainY), axis=0) 
    return resultX, resultY

In [ ]:
(augTrainX,augTrainY) = PrepareAugmentedData(trainX,trainY) 

In [ ]:
print(augTrainX.shape)
print(augTrainY.shape)

In [ ]:
# define the model
model_with_conv = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation=tf.nn.relu, input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Conv2D(128, (3,3), padding='same', activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Flatten(),    
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    #  10-node softmax layer, with each node representing a class of clothing.
    tf.keras.layers.Dense(10,  activation=tf.nn.softmax)
])
 
# compile the model
model_with_conv.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
print(model_with_conv.summary())

In [ ]:
    from tensorflow.keras.callbacks import Callback
    logs = Callback()

In [ ]:
model_with_conv.fit(trainX, trainY,verbose=1, epochs=50,callbacks=[logs],validation_data=(testX,testY))


In [ ]:
def plot_image(i, predictions_array, true_labels, images):
  predictions_array, true_label, img = predictions_array[i], true_labels[i], images[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
   
  plt.imshow(img[...,0], cmap=plt.cm.binary)
 
  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'
   
  plt.xlabel("{} {:2.0f}% ({})".format(class_object[predicted_label],
                                100*np.max(predictions_array),
                                class_object[true_label]),
                                color=color)


In [ ]:
num_rows = 3
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*num_cols, 2*num_rows))
test_images = testX[:num_images]
predictions = model_with_conv.predict(testX)
for i in range(num_images):    
    plt.subplot(num_rows, num_cols, i+1)   
    plot_image(i, predictions, testY, testX)    
 
plt.tight_layout()
plt.show()

In [ ]:
acc = model_with_conv.history.history['accuracy']
val_accuracy = model_with_conv.history.history['val_accuracy']
loss  =  model_with_conv.history.history['loss']
val_loss = model_with_conv.history.history['val_loss']


In [ ]:
def GenerateGraphs(acc, val_acc, loss, val_loss):
     epochs = range(len(acc))
     plt.plot(epochs, acc, 'bo', label='Training accuracy')
     plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
     plt.title('Training and validation accuracy')
     plt.figure()
     plt.plot(epochs, loss, 'bo', label='Training Loss')
     plt.plot(epochs, val_loss, 'b', label='Validation Loss')
     plt.title('Training and validation loss')
     plt.legend()
     plt.show()
    

In [ ]:
GenerateGraphs(acc,val_accuracy,loss,val_loss)